In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [2]:
# Generating a sample dataset with multicollinearity
np.random.seed(0)
size = 300 # Increase sample size for better effect of Ridge regression


In [3]:
# Generate predictors X1, X2, X3 where X2 is highly correlated with X1 and X3 is somewhat correlated with both
X1 = np.random.normal(0, 1, size)
X2 = X1 + np.random.normal(0, 0.1, size) # X2 is highly correlated with X1
X3 = 0.5 * X1 + 0.5 * X2 + np.random.normal(0, 0.1, size) # X3 is correlated with X1 and X2

In [4]:
# Generate a response variable with some noise
Y = 2 * X1 + 3 * X2 + 1.5 * X3 + np.random.normal(0, 1, size)

In [5]:
# Combine into a Dataframe
df = pd.DataFrame({
    'X1': X1,
    'X2': X2,
    'X3': X3,
    'Y': Y
})
print(df)

           X1        X2        X3          Y
0    1.764052  1.633400  1.543683   9.282096
1    0.400157  0.565970  0.524796   2.601979
2    0.978738  0.966922  0.878393   6.543375
3    2.240893  2.172875  2.230695  14.536766
4    1.867558  1.934196  1.760281  11.326397
..        ...       ...       ...        ...
295  1.136891  1.174815  1.355649   8.869286
296  0.097725  0.050722 -0.011432   0.349259
297  0.582954  0.561281  0.417958   2.882909
298 -0.399449 -0.492465 -0.186514  -4.567944
299  0.370056  0.352197  0.320723   2.867491

[300 rows x 4 columns]


In [6]:
# Calculating VIFs to show multicollinearity
vif_data = pd.DataFrame()
vif_data["feature"] = df.columns[:-1]
vif_data["VIF"] = [
    variance_inflation_factor(df.values, i) for i in range(df.shape[1] - 1)
]

print(vif_data)

  feature         VIF
0      X1  119.686869
1      X2  157.409395
2      X3  111.435351


In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df[['X1', 'X2', 'X3']],
    df['Y'],
    test_size=0.2,
    random_state=42
)
print("X_Train: ", X_train)
print("\nX_Test: ", X_test)
print("\nY_Train: ", y_train)
print("\nY_Test: ",y_test)

X_Train:             X1        X2        X3
232 -0.542861 -0.420374 -0.478558
59  -0.362741 -0.300705 -0.426285
6    0.950088  0.816663  0.872327
185 -1.602058 -1.867620 -1.719994
173 -0.651026 -0.721998 -0.667237
..        ...       ...       ...
188 -0.739563 -0.744133 -0.877826
71   0.128983  0.029671 -0.109132
106 -0.413619 -0.332167 -0.279248
270  0.399046  0.444425  0.529873
102 -1.270485 -1.193819 -1.311019

[240 rows x 3 columns]

X_Test:             X1        X2        X3
203  0.655264  0.631842  0.604431
266  0.523891  0.559367  0.530151
152 -0.744755 -0.698442 -0.676779
9    0.410599  0.394641  0.364705
233  0.416050  0.422536  0.412329
226  0.156507  0.250181  0.236347
196  0.771791  0.705935  0.746322
109  1.480515  1.399750  1.461282
5   -0.977278 -1.023350 -1.059320
175  0.681595  0.781034  0.552181
237 -2.069985 -2.088210 -2.111664
57   0.302472  0.254674  0.146734
218  2.259309  2.287737  2.188264
45  -0.438074 -0.466110 -0.482588
182 -0.643618 -0.734495 -0.627864
221 

In [8]:
# Fit a standard linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

print(y_pred_lr)

[  4.21496877   3.66124044  -4.51359484   2.65475603   2.82502408
   1.57984004   4.81865917   9.38582048  -6.56338024   4.83567875
 -13.53775891   1.71146809  14.88352385  -2.93322178  -4.48536211
  -2.60117249   7.06982401   2.86458179  -9.0399439    6.63074441
  -1.91189081  12.4128052   -9.77792297   3.32272567  -8.38122222
 -11.27601967   6.34194438   7.86981514  -8.06511215   2.8157149
  -1.35727257  -8.08043521 -13.43229567  14.31631762  -2.34496279
   6.86325718  -1.69156138  -2.24650421  -8.16502857  -8.17443906
   5.53094759  -5.38041437  -6.74968367  -4.05960022  -4.33708983
 -11.00920077  -8.05721644  -1.1065408    6.43915395  -2.85077327
  -1.7212783   15.97867943   4.48034707  -4.11308971   3.64412053
 -13.22205507  12.20362765   1.13837627   0.26493609  -9.66587509]


In [10]:
# Fit a Ridge regression model with a higher alpha for a better effect on multicollinearity
ridge_model = Ridge(alpha=100)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

print(y_pred_ridge)

[  3.69848667   3.17208462  -3.90611045   2.33063074   2.48422212
   1.33310721   4.32743822   8.34132345  -5.68875625   3.93636249
 -11.77384864   1.44753521  12.88291295  -2.51672538  -3.69298701
  -2.11455952   6.49320015   2.43199552  -7.93938343   5.3979287
  -1.45058344  10.85424594  -8.40241385   2.6364092   -7.02178253
  -9.59384226   5.54393061   7.06348505  -6.7932747    2.56733953
  -1.07436165  -6.86538741 -11.56187398  12.61936628  -1.84221417
   5.74589551  -1.44687389  -1.9557473   -6.92323083  -7.15382375
   4.86919102  -4.83268447  -5.7179748   -3.68769601  -3.43787023
  -9.58971571  -6.81994251  -1.08920508   5.62287273  -2.39034158
  -1.3992625   13.77301116   3.98957043  -3.56677268   3.07552206
 -11.4647177   10.44660081   0.91280664   0.03518101  -8.47947336]


In [11]:
# Calculate the performance
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

In [12]:
# Output the results
print("Variance Inflation Factor (VIF):")
print(vif_data)
print("\nLinear Regression - MSE: {:.2f}, R2: {:.3f}".format(mse_lr, r2_lr))
print("Ridge Regression - MSE: {:.2f}, R2: {:.3f}".format(mse_ridge, r2_ridge))

Variance Inflation Factor (VIF):
  feature         VIF
0      X1  119.686869
1      X2  157.409395
2      X3  111.435351

Linear Regression - MSE: 0.86, R2: 0.985
Ridge Regression - MSE: 1.98, R2: 0.965
